In [23]:
# Now I want to examine if there is statistical significance between the prevelance of cideries
# in each cluster

# load pandas
import pandas as pd

# import data files 
city_cluster_0 = pd.read_csv('data/cidery_cluster_0.csv')
city_cluster_1 = pd.read_csv('data/cidery_cluster_1.csv')
city_cluster_2 = pd.read_csv('data/cidery_cluster_2.csv')

In [24]:
# write a function to create permutation samples 

# load numpy
import numpy as np

def permutation_sample(data1, data2, data3):
    """Generate a permutation sample from three datasets"""
    
    # concatenate the data series
    data = np.concatenate((data1, data2, data3))
    
    # permute the array
    permuted_sample = np.random.permutation(data)
    
    # split the array into three samples with the same lengths as the original datasets
    len1 = len(data1)
    len2 = len(data2) + len1
    perm_sample_1 = permuted_sample[:len1]
    perm_sample_2 = permuted_sample[len1:len2]
    perm_sample_3 = permuted_sample[len2:]
    
    return perm_sample_1, perm_sample_2, perm_sample_3
    
a, b, c = permutation_sample(city_cluster_0.Frequency, city_cluster_1.Frequency, city_cluster_2.Frequency)

In [25]:
# write a function to create multiple permutation samples 

def draw_perm_reps(data1, data2, data3, func, size = 1):
    """Generate multiple permutation replicates."""
    
    # intialize an array of replicates
    perm_replicates = np.empty(size)
    
    # generate a permutation sample
    for i in range(size):
        perm1_sample, perm2_sample, perm3_sample = permutation_sample(data1, data2, data3)
        
        # compute test statistic
        perm_replicates[i] = func(perm1_sample, perm2_sample, perm3_sample)
        
    return perm_replicates
        

In [ ]:
# 3 way hypothesis testing? 
